In [2862]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhinaba1biswas/esophageal-cancer-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/abhinaba1biswas/esophageal-cancer-dataset/versions/1


In [2863]:
import os
os.listdir(path)

['Esophageal_Dataset.csv']

In [2864]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2865]:
df = pd.read_csv(f'{path}/Esophageal_Dataset.csv')
df.head()

,Unnamed: 0,patient_barcode,tissue_source_site,patient_id,bcr_patient_uuid,informed_consent_verified,icd_o_3_site,icd_o_3_histology,icd_10,tissue_prospective_collection_indicator,...,primary_pathology_lymph_node_examined_count,primary_pathology_number_of_lymphnodes_positive_by_he,primary_pathology_number_of_lymphnodes_positive_by_ihc,primary_pathology_planned_surgery_status,primary_pathology_treatment_prior_to_surgery,primary_pathology_residual_tumor,primary_pathology_karnofsky_performance_score,primary_pathology_eastern_cancer_oncology_group,primary_pathology_radiation_therapy,primary_pathology_postoperative_rx_tx
0,0,TCGA-2H-A9GF,2H,A9GF,0500F1A6-A528-43F3-B035-12D3B7C99C0F,YES,C15.5,8140/3,C15.5,NO,...,8.0,7.0,0.0,NaN,NaN,R1,NaN,NaN,NO,NO
1,1,TCGA-2H-A9GG,2H,A9GG,70084008-697D-442D-8F74-C12F8F598570,YES,C15.5,8140/3,C15.5,NO,...,19.0,4.0,0.0,NaN,NaN,R1,NaN,NaN,NO,NO
2,2,TCGA-2H-A9GH,2H,A9GH,606DC5B8-7625-42A6-A936-504EF25623A4,YES,C15.5,8140/3,C15.5,NO,...,30.0,1.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO
3,3,TCGA-2H-A9GI,2H,A9GI,CEAF98F8-517E-457A-BF29-ACFE22893D49,YES,C15.5,8140/3,C15.5,NO,...,8.0,4.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO
4,4,TCGA-2H-A9GJ,2H,A9GJ,EE47CD59-C8D8-4B1E-96DB-91C679E4106F,YES,C15.5,8140/3,C15.5,NO,...,19.0,0.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO


In [2866]:
df.shape

(3985, 85)

In [2867]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3985 entries, 0 to 3984
Data columns (total 85 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Unnamed: 0                                                     3985 non-null   int64  
 1   patient_barcode                                                3985 non-null   object 
 2   tissue_source_site                                             3985 non-null   object 
 3   patient_id                                                     3985 non-null   object 
 4   bcr_patient_uuid                                               3985 non-null   object 
 5   informed_consent_verified                                      3985 non-null   object 
 6   icd_o_3_site                                                   3985 non-null   object 
 7   icd_o_3_histology                                           

In [2868]:
df['primary_pathology_postoperative_rx_tx'].isnull().sum()

np.int64(658)

In [2869]:

columns_before = df.columns

# 计算每列的缺失值数量并删除缺失值大于50的列
nan_counts = df.isnull().sum()
df = df.loc[:, nan_counts <= 300]

# 获取删除后的列名
columns_after = df.columns

# 找出被删除的列
deleted_columns = set(columns_before) - set(columns_after)

# 打印被删除的列
print("Deleted columns:", deleted_columns)


Deleted columns: {'days_to_death', 'stage_event_igcccg_stage', 'stage_event_pathologic_stage', 'primary_pathology_number_of_lymphnodes_positive_by_he', 'primary_pathology_karnofsky_performance_score', 'person_neoplasm_cancer_status', 'stage_event_gleason_grading', 'primary_pathology_eastern_cancer_oncology_group', 'stopped_smoking_year', 'primary_pathology_columnar_mucosa_goblet_cell_present', 'primary_pathology_treatment_prior_to_surgery', 'age_began_smoking_in_years', 'primary_pathology_number_of_lymphnodes_positive_by_ihc', 'reflux_history', 'primary_pathology_primary_lymph_node_presentation_assessment', 'race_list', 'goblet_cells_present', 'stage_event_masaoka_stage', 'primary_pathology_lymph_node_metastasis_radiographic_evidence', 'ethnicity', 'stage_event_psa', 'country_of_birth', 'stage_event_serum_markers', 'days_to_last_followup', 'state_province_of_procurement', 'amount_of_alcohol_consumption_per_day', 'primary_pathology_radiation_therapy', 'primary_pathology_lymph_node_exami

In [2870]:
df.shape

(3985, 39)

In [2871]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(3506, 39)

In [2872]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3506 entries, 0 to 3505
Data columns (total 39 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Unnamed: 0                                              3506 non-null   int64  
 1   patient_barcode                                         3506 non-null   object 
 2   tissue_source_site                                      3506 non-null   object 
 3   patient_id                                              3506 non-null   object 
 4   bcr_patient_uuid                                        3506 non-null   object 
 5   informed_consent_verified                               3506 non-null   object 
 6   icd_o_3_site                                            3506 non-null   object 
 7   icd_o_3_histology                                       3506 non-null   object 
 8   icd_10                                

In [2873]:
df.columns

Index(['Unnamed: 0', 'patient_barcode', 'tissue_source_site', 'patient_id',
       'bcr_patient_uuid', 'informed_consent_verified', 'icd_o_3_site',
       'icd_o_3_histology', 'icd_10',
       'tissue_prospective_collection_indicator',
       'tissue_retrospective_collection_indicator', 'days_to_birth', 'gender',
       'height', 'weight', 'country_of_procurement', 'other_dx',
       'history_of_neoadjuvant_treatment', 'vital_status',
       'alcohol_history_documented', 'has_new_tumor_events_information',
       'day_of_form_completion', 'month_of_form_completion',
       'year_of_form_completion', 'has_follow_ups_information',
       'has_drugs_information', 'has_radiations_information', 'project',
       'stage_event_system_version', 'stage_event_tnm_categories',
       'primary_pathology_tumor_tissue_site',
       'primary_pathology_esophageal_tumor_cental_location',
       'primary_pathology_esophageal_tumor_involvement_sites',
       'primary_pathology_histological_type',
       

In [2874]:
df.head()

,Unnamed: 0,patient_barcode,tissue_source_site,patient_id,bcr_patient_uuid,informed_consent_verified,icd_o_3_site,icd_o_3_histology,icd_10,tissue_prospective_collection_indicator,...,stage_event_tnm_categories,primary_pathology_tumor_tissue_site,primary_pathology_esophageal_tumor_cental_location,primary_pathology_esophageal_tumor_involvement_sites,primary_pathology_histological_type,primary_pathology_neoplasm_histologic_grade,primary_pathology_days_to_initial_pathologic_diagnosis,primary_pathology_age_at_initial_pathologic_diagnosis,primary_pathology_year_of_initial_pathologic_diagnosis,primary_pathology_initial_pathologic_diagnosis_method
0,0,TCGA-2H-A9GF,2H,A9GF,0500F1A6-A528-43F3-B035-12D3B7C99C0F,YES,C15.5,8140/3,C15.5,NO,...,T3N1M0,Esophagus,Distal,Distal,"Esophagus Adenocarcinoma, NOS",G3,0,67,2001.0,"Other method, specify:"
1,1,TCGA-2H-A9GG,2H,A9GG,70084008-697D-442D-8F74-C12F8F598570,YES,C15.5,8140/3,C15.5,NO,...,T3N1M0,Esophagus,Distal,Distal,"Esophagus Adenocarcinoma, NOS",G2,0,66,1999.0,"Other method, specify:"
2,2,TCGA-2H-A9GH,2H,A9GH,606DC5B8-7625-42A6-A936-504EF25623A4,YES,C15.5,8140/3,C15.5,NO,...,T1N1M0,Esophagus,Distal,Distal,"Esophagus Adenocarcinoma, NOS",G2,0,44,1998.0,"Other method, specify:"
3,3,TCGA-2H-A9GI,2H,A9GI,CEAF98F8-517E-457A-BF29-ACFE22893D49,YES,C15.5,8140/3,C15.5,NO,...,T3N1M0,Esophagus,Distal,Distal,"Esophagus Adenocarcinoma, NOS",G2,0,68,1999.0,"Other method, specify:"
4,4,TCGA-2H-A9GJ,2H,A9GJ,EE47CD59-C8D8-4B1E-96DB-91C679E4106F,YES,C15.5,8140/3,C15.5,NO,...,T1N0M0,Esophagus,Distal,Distal,"Esophagus Adenocarcinoma, NOS",G2,0,57,2000.0,"Other method, specify:"


In [2875]:
df.drop(['Unnamed: 0', 'patient_barcode', 'patient_id', 'bcr_patient_uuid', 'informed_consent_verified', 'history_of_neoadjuvant_treatment', 'project', 'stage_event_tnm_categories', 'primary_pathology_tumor_tissue_site', 'primary_pathology_days_to_initial_pathologic_diagnosis','primary_pathology_initial_pathologic_diagnosis_method'
], axis=1, inplace=True)

In [2876]:
df['tissue_source_site'].unique()

array(['2H', 'IC', 'IG', 'JY', 'KH', 'L5', 'L7', 'LN', 'Q9', 'R6', 'RE',
       'S8', 'VR', 'Z6'], dtype=object)

In [2877]:
enc = OneHotEncoder(categories='auto')
tss = df['tissue_source_site'].values.reshape(-1, 1)
enc.fit(tss)
new_features = enc.get_feature_names_out()
print(new_features)
new_tss = pd.DataFrame(enc.transform(tss).toarray())

['x0_2H' 'x0_IC' 'x0_IG' 'x0_JY' 'x0_KH' 'x0_L5' 'x0_L7' 'x0_LN' 'x0_Q9'
 'x0_R6' 'x0_RE' 'x0_S8' 'x0_VR' 'x0_Z6']


In [2878]:
new_tss.columns = ['2H', 'IC', 'IG', 'JY', 'KH', 'L5', 'L7', 'LN', 'Q9', 'R6', 'RE', 'S8', 'VR', 'Z6']

In [2879]:
df = pd.concat([df, new_tss], axis=1)
df.drop('tissue_source_site', axis=1, inplace=True)

In [2880]:
df.head()

,icd_o_3_site,icd_o_3_histology,icd_10,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,country_of_procurement,...,KH,L5,L7,LN,Q9,R6,RE,S8,VR,Z6
0,C15.5,8140/3,C15.5,NO,YES,-24487,MALE,183.0,95.0,Netherlands,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C15.5,8140/3,C15.5,NO,YES,-24328,MALE,178.0,74.0,Netherlands,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C15.5,8140/3,C15.5,NO,YES,-16197,MALE,183.0,91.0,Netherlands,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C15.5,8140/3,C15.5,NO,YES,-25097,MALE,188.0,100.0,Netherlands,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C15.5,8140/3,C15.5,NO,YES,-21180,MALE,189.0,70.0,Netherlands,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2881]:
enc2 = OneHotEncoder(categories='auto')
icd = df['icd_o_3_site'].values.reshape(-1, 1)
enc2.fit(icd)
new_features = enc2.get_feature_names_out()
print(new_features)
new_icd = pd.DataFrame(enc2.transform(icd).toarray()) 

['x0_C15.1' 'x0_C15.3' 'x0_C15.4' 'x0_C15.5' 'x0_C15.9' 'x0_C16.0']


In [2882]:
new_icd.columns = ['C15.1', 'C15.3', 'C15.4', 'C15.5', 'C15.9', 'C16.0']

In [2883]:
df = pd.concat([df, new_icd], axis=1)
df.drop('icd_o_3_site', axis=1, inplace=True)

In [2884]:
df.head()

,icd_o_3_histology,icd_10,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,country_of_procurement,other_dx,...,RE,S8,VR,Z6,C15.1,C15.3,C15.4,C15.5,C15.9,C16.0
0,8140/3,C15.5,NO,YES,-24487,MALE,183.0,95.0,Netherlands,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,8140/3,C15.5,NO,YES,-24328,MALE,178.0,74.0,Netherlands,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,8140/3,C15.5,NO,YES,-16197,MALE,183.0,91.0,Netherlands,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,8140/3,C15.5,NO,YES,-25097,MALE,188.0,100.0,Netherlands,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,8140/3,C15.5,NO,YES,-21180,MALE,189.0,70.0,Netherlands,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [2885]:
df['icd_o_3_histology'].unique()

array(['8140/3', '8070/3', '8071/3', '8480/3', '8211/3'], dtype=object)

In [2886]:
enc3 = OneHotEncoder(categories='auto')
icdhis = df['icd_o_3_histology'].values.reshape(-1, 1)
enc3.fit(icdhis)
new_features = enc3.get_feature_names_out()
print(new_features)
new_icdhis = pd.DataFrame(enc3.transform(icdhis).toarray())

['x0_8070/3' 'x0_8071/3' 'x0_8140/3' 'x0_8211/3' 'x0_8480/3']


In [2887]:
new_icdhis.columns = ['8070/3', '8071/3', '8140/3', '8211/3', '8480/3']

In [2888]:
df = pd.concat([df, new_icdhis], axis=1)
df.drop('icd_o_3_histology', axis=1, inplace=True)

In [2889]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3506 entries, 0 to 3505
Data columns (total 50 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   icd_10                                                  3506 non-null   object 
 1   tissue_prospective_collection_indicator                 3506 non-null   object 
 2   tissue_retrospective_collection_indicator               3506 non-null   object 
 3   days_to_birth                                           3506 non-null   int64  
 4   gender                                                  3506 non-null   object 
 5   height                                                  3506 non-null   float64
 6   weight                                                  3506 non-null   float64
 7   country_of_procurement                                  3506 non-null   object 
 8   other_dx                              

In [2890]:
df.columns

Index(['icd_10', 'tissue_prospective_collection_indicator',
       'tissue_retrospective_collection_indicator', 'days_to_birth', 'gender',
       'height', 'weight', 'country_of_procurement', 'other_dx',
       'vital_status', 'alcohol_history_documented',
       'has_new_tumor_events_information', 'day_of_form_completion',
       'month_of_form_completion', 'year_of_form_completion',
       'has_follow_ups_information', 'has_drugs_information',
       'has_radiations_information', 'stage_event_system_version',
       'primary_pathology_esophageal_tumor_cental_location',
       'primary_pathology_esophageal_tumor_involvement_sites',
       'primary_pathology_histological_type',
       'primary_pathology_neoplasm_histologic_grade',
       'primary_pathology_age_at_initial_pathologic_diagnosis',
       'primary_pathology_year_of_initial_pathologic_diagnosis', '2H', 'IC',
       'IG', 'JY', 'KH', 'L5', 'L7', 'LN', 'Q9', 'R6', 'RE', 'S8', 'VR', 'Z6',
       'C15.1', 'C15.3', 'C15.4', 'C15.

In [2891]:
df.head()

,icd_10,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,country_of_procurement,other_dx,vital_status,...,C15.3,C15.4,C15.5,C15.9,C16.0,8070/3,8071/3,8140/3,8211/3,8480/3
0,C15.5,NO,YES,-24487,MALE,183.0,95.0,Netherlands,No,Dead,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,C15.5,NO,YES,-24328,MALE,178.0,74.0,Netherlands,No,Dead,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,C15.5,NO,YES,-16197,MALE,183.0,91.0,Netherlands,No,Dead,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,C15.5,NO,YES,-25097,MALE,188.0,100.0,Netherlands,No,Dead,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,C15.5,NO,YES,-21180,MALE,189.0,70.0,Netherlands,No,Dead,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [2892]:
df['icd_10'].unique()

array(['C15.5', 'C15.9', 'C15.4', 'C15.3', 'C16.0'], dtype=object)

In [2893]:
enc4 = OneHotEncoder(categories='auto')
icd10 = df['icd_10'].values.reshape(-1, 1)
enc4.fit(icd10)
new_features = enc4.get_feature_names_out()
print(new_features)
new_icd10 = pd.DataFrame(enc4.transform(icd10).toarray())

['x0_C15.3' 'x0_C15.4' 'x0_C15.5' 'x0_C15.9' 'x0_C16.0']


In [2894]:
new_icd10.columns = ['C15.3', 'C15.4', 'C15.5', 'C15.9', 'C16.0']

In [2895]:
df = pd.concat([df, new_icd10], axis=1)
df.drop('icd_10', axis=1, inplace=True)

In [2896]:
df.head()

,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,country_of_procurement,other_dx,vital_status,alcohol_history_documented,...,8070/3,8071/3,8140/3,8211/3,8480/3,C15.3,C15.4,C15.5,C15.9,C16.0
0,NO,YES,-24487,MALE,183.0,95.0,Netherlands,No,Dead,NO,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NO,YES,-24328,MALE,178.0,74.0,Netherlands,No,Dead,NO,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,NO,YES,-16197,MALE,183.0,91.0,Netherlands,No,Dead,NO,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,NO,YES,-25097,MALE,188.0,100.0,Netherlands,No,Dead,NO,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NO,YES,-21180,MALE,189.0,70.0,Netherlands,No,Dead,NO,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [2897]:
df['tissue_prospective_collection_indicator'] = df['tissue_prospective_collection_indicator'].map({'NO': 0, 'YES': 1})
df['tissue_retrospective_collection_indicator'] = df['tissue_retrospective_collection_indicator'].map({'NO': 0, 'YES': 1})
df['gender'] = df['gender'].map({'FEMALE': 0, 'MALE': 1})



In [2898]:
df['country_of_procurement'].unique()

array(['Netherlands', 'United States', 'Russia', 'Ukraine', 'Vietnam',
       'Bulgaria', 'Canada', 'Australia', 'Brazil'], dtype=object)

In [2899]:
enc = OneHotEncoder(categories='auto')
country = df['country_of_procurement'].values.reshape(-1, 1)
enc.fit(country)
new_features = enc.get_feature_names_out()
print(new_features)
new_country = pd.DataFrame(enc.transform(country).toarray())

['x0_Australia' 'x0_Brazil' 'x0_Bulgaria' 'x0_Canada' 'x0_Netherlands'
 'x0_Russia' 'x0_Ukraine' 'x0_United States' 'x0_Vietnam']


In [2900]:
new_country.columns = ['Australia', 'Brazil', 'Bulgaria', 'Canada', 'Netherlands', 'Russia', 'Ukraine', 'United States', 'Vietnam'] 

In [2901]:
df = pd.concat([df, new_country], axis=1)
df.drop('country_of_procurement', axis=1, inplace=True)

In [2902]:
df.head()

,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,other_dx,vital_status,alcohol_history_documented,has_new_tumor_events_information,...,C16.0,Australia,Brazil,Bulgaria,Canada,Netherlands,Russia,Ukraine,United States,Vietnam
0,0,1,-24487,1,183.0,95.0,No,Dead,NO,YES,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,1,-24328,1,178.0,74.0,No,Dead,NO,YES,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,1,-16197,1,183.0,91.0,No,Dead,NO,YES,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0,1,-25097,1,188.0,100.0,No,Dead,NO,YES,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,1,-21180,1,189.0,70.0,No,Dead,NO,YES,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [2903]:
df['alcohol_history_documented'].unique()

array(['NO', 'YES'], dtype=object)

In [2904]:
df['other_dx']= df['other_dx'].map({'No': 0, 'Yes': 1})
df['vital_status']= df['vital_status'].map({'Dead': 0, 'Alive': 1})

df['alcohol_history_documented']= df['alcohol_history_documented'].map({'NO': 0, 'YES': 1})

df['has_new_tumor_events_information']= df['has_new_tumor_events_information'].map({'NO': 0, 'YES': 1})

df['has_follow_ups_information'] = df['has_follow_ups_information'].map({'NO':0, 'YES':1})

df['has_drugs_information'] = df['has_drugs_information'].map({'NO':0, 'YES':1})

df['has_radiations_information'] = df['has_radiations_information'].map({'NO':0, 'YES':1})


df['stage_event_system_version'] = df['stage_event_system_version'].map({'5th':0, '6th': 1, '7th': 2})



In [2905]:
df['has_new_tumor_events_information'].unique()

array([1, 0])

In [2906]:
df.head()

,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,days_to_birth,gender,height,weight,other_dx,vital_status,alcohol_history_documented,has_new_tumor_events_information,...,C16.0,Australia,Brazil,Bulgaria,Canada,Netherlands,Russia,Ukraine,United States,Vietnam
0,0,1,-24487,1,183.0,95.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,1,-24328,1,178.0,74.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,1,-16197,1,183.0,91.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0,1,-25097,1,188.0,100.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,1,-21180,1,189.0,70.0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [2907]:
df.columns

Index(['tissue_prospective_collection_indicator',
       'tissue_retrospective_collection_indicator', 'days_to_birth', 'gender',
       'height', 'weight', 'other_dx', 'vital_status',
       'alcohol_history_documented', 'has_new_tumor_events_information',
       'day_of_form_completion', 'month_of_form_completion',
       'year_of_form_completion', 'has_follow_ups_information',
       'has_drugs_information', 'has_radiations_information',
       'stage_event_system_version',
       'primary_pathology_esophageal_tumor_cental_location',
       'primary_pathology_esophageal_tumor_involvement_sites',
       'primary_pathology_histological_type',
       'primary_pathology_neoplasm_histologic_grade',
       'primary_pathology_age_at_initial_pathologic_diagnosis',
       'primary_pathology_year_of_initial_pathologic_diagnosis', '2H', 'IC',
       'IG', 'JY', 'KH', 'L5', 'L7', 'LN', 'Q9', 'R6', 'RE', 'S8', 'VR', 'Z6',
       'C15.1', 'C15.3', 'C15.4', 'C15.5', 'C15.9', 'C16.0', '8070/3',
    

In [2908]:
df['primary_pathology_histological_type'] =df['primary_pathology_histological_type'].map({'Esophagus Adenocarcinoma, NOS': 0, 'Esophagus Squamous Cell Carcinoma': 1})

df['primary_pathology_neoplasm_histologic_grade'] =df['primary_pathology_neoplasm_histologic_grade'].map({'GX': 0, 'G1': 1, 'G2': 2, 'G3': 3})

In [2909]:
enc = OneHotEncoder(categories='auto')
ppetcl = df['primary_pathology_esophageal_tumor_cental_location'].values.reshape(-1, 1)
enc.fit(ppetcl)
new_features = enc.get_feature_names_out()
print(new_features)
new_ppetcl = pd.DataFrame(enc.transform(ppetcl).toarray())

['x0_Distal' 'x0_Mid' 'x0_Proximal']


In [2910]:
new_ppetcl.columns = ['Distal', 'Mid', 'Proximal']

In [2911]:
df = pd.concat([df, new_ppetcl], axis=1)
df.drop('primary_pathology_esophageal_tumor_cental_location', axis=1, inplace=True)

In [2912]:
enc = OneHotEncoder(categories='auto')
ppetis = df['primary_pathology_esophageal_tumor_involvement_sites'].values.reshape(-1, 1)
enc.fit(ppetis)
new_features = enc.get_feature_names_out()
print(new_features)
new_ppetis = pd.DataFrame(enc.transform(ppetis).toarray())

['x0_Distal' 'x0_Mid' 'x0_MidDistal' 'x0_Proximal' 'x0_ProximalMid']


In [2913]:
new_ppetis.columns = ['Distal1', 'Mid1','MidDistal1', 'Proximal1', 'ProximalMid1']

In [2914]:
df = pd.concat([df, new_ppetis], axis=1)
df.drop('primary_pathology_esophageal_tumor_involvement_sites', axis=1, inplace=True)

In [2915]:
y = df['vital_status'].values
X = df.drop('vital_status', axis=1)

In [2916]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(3506, 67)
(3506, 34)


In [2917]:
# build mask
gender_0_mask = df['gender'] == 0
gender_1_mask = df['gender'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  556
Male:  2950


In [2918]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [2919]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
Processing fold 4 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KN

In [2920]:
results_df = pd.concat(results_list, ignore_index=True)

In [2921]:
result_path = './results/K35_result.xlsx'
results_df.to_excel(result_path, index=False)

In [2922]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,1.0,1,0,0.0,46,10,0,0,...,0,0,0.695652,1,0,0.304348,32,10,0,14
1,2,Female,1.0,1,0,0.0,43,13,0,0,...,0,0,0.674419,1,0,0.325581,29,13,0,14
2,3,Female,1.0,1,0,0.0,44,12,0,0,...,0,0,0.636364,1,0,0.363636,28,12,0,16
3,4,Female,1.0,1,0,0.0,49,7,0,0,...,0,0,0.673469,1,0,0.326531,33,7,0,16
4,5,Female,1.0,1,0,0.0,49,7,0,0,...,0,0,0.612245,1,0,0.387755,30,7,0,19


In [2923]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(100.0), pvalue=np.float64(6.340287417294021e-05))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(0.0), pvalue=np.float64(5.469864163379713e-05))
DT -TPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
DT - FPR: MannwhitneyuResult(statistic=np.float64(45.0), pvalue=np.float64(0.36812025069351895))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
RF -TPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
RF - FPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
LR -TPR: MannwhitneyuResult(statistic=np.float64(60.0), pvalue=np.float64(0.16807831903497028))
LR - FPR: MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(1.0))
LR - FN/FP: MannwhitneyuResult(stati